In [71]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Loan_Binning.pickle')

def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['Loan_Status'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

dt_train = pd.DataFrame()
dt_train['Loan_Status'] = data['Loan_Status']
ATTRS = ['Education','Self_Employed','Credit_History','Property_Area','APPLICANTINCOME_BIN','COAPPLICANTINCOME_BIN','LOANAMOUNT_BIN','LOAN_AMOUNT_TERM_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train

WOE Education -> {0: -0.30047255311614995, 1: 0.07907968599641366}
WOE Self_Employed -> {0: -0.18796404419303647, 1: 0.03150977823221087}
WOE Credit_History -> {0.0: -3.2816767261072313, 1.0: 0.6041439069268907}
WOE Property_Area -> {1: -0.10307480037697031, 2: -0.437494350680441, 3: 0.46782734982314}
WOE APPLICANTINCOME_BIN -> {Interval(5417.0, 6460.0, closed='right'): 0.1823215567939548, Interval(149.999, 2232.2, closed='right'): -0.01834913866819643, Interval(6460.0, 9883.1, closed='right'): 0.4227069145426705, Interval(9883.1, 81000.0, closed='right'): -0.3992731378602432, Interval(3887.5, 4435.8, closed='right'): 0.11952065555492423, Interval(3418.0, 3887.5, closed='right'): -0.6953234599825407, Interval(3031.3, 3418.0, closed='right'): -0.01834913866819643, Interval(2589.8, 3031.3, closed='right'): 0.2657031657330057, Interval(2232.2, 2589.8, closed='right'): 0.4227069145426705, Interval(4435.8, 5417.0, closed='right'): -0.10536051565782616}
WOE COAPPLICANTINCOME_BIN -> {Interval

,Loan_Status,Education_WOE,Self_Employed_WOE,Credit_History_WOE,Property_Area_WOE,APPLICANTINCOME_BIN_WOE,COAPPLICANTINCOME_BIN_WOE,LOANAMOUNT_BIN_WOE,LOAN_AMOUNT_TERM_BIN_WOE
2,1,0.079080,-0.187964,0.604144,-0.103075,0.265703,-0.080500,-0.149812,0.052510
3,1,-0.300473,0.031510,0.604144,-0.103075,0.422707,0.930451,0.011307,0.052510
4,1,0.079080,0.031510,0.604144,-0.103075,0.182322,-0.080500,0.265703,0.052510
5,1,0.079080,-0.187964,0.604144,-0.103075,-0.105361,-0.519559,-0.638165,0.052510
6,1,-0.300473,0.031510,0.604144,-0.103075,0.422707,0.265703,0.160343,0.052510
...,...,...,...,...,...,...,...,...,...
494,0,0.079080,0.031510,-3.281677,-0.437494,-0.695323,0.930451,0.011307,0.052510
495,1,0.079080,0.031510,0.604144,0.467827,-0.018349,-0.080500,-0.196332,-0.021979
496,1,-0.300473,0.031510,0.604144,-0.437494,0.265703,0.265703,-0.196332,0.052510
498,1,0.079080,-0.187964,0.604144,0.467827,0.265703,-0.080500,0.160343,0.052510


In [72]:
Y =  dt_train['Loan_Status'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['Self_Employed_WOE'].to_numpy()
x_3 = dt_train['Credit_History_WOE'].to_numpy()
x_4 = dt_train['Property_Area_WOE'].to_numpy()
x_5 = dt_train['APPLICANTINCOME_BIN_WOE'].to_numpy()
x_6 = dt_train['COAPPLICANTINCOME_BIN_WOE'].to_numpy()
x_7 = dt_train['LOANAMOUNT_BIN_WOE'].to_numpy()
x_8 = dt_train['LOAN_AMOUNT_TERM_BIN_WOE'].to_numpy()


array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8])
X = np.transpose(X)
X

array([[ 1.        ,  0.07907969, -0.18796404, ..., -0.08050048,
        -0.14981228,  0.05250995],
       [ 1.        , -0.30047255,  0.03150978, ...,  0.93045087,
         0.01130707,  0.05250995],
       [ 1.        ,  0.07907969,  0.03150978, ..., -0.08050048,
         0.26570317,  0.05250995],
       ...,
       [ 1.        , -0.30047255,  0.03150978, ...,  0.26570317,
        -0.19633229,  0.05250995],
       [ 1.        ,  0.07907969, -0.18796404, ..., -0.08050048,
         0.16034265,  0.05250995],
       [ 1.        ,  0.07907969,  0.03150978, ..., -0.5195593 ,
         0.26570317,  0.05250995]])

In [73]:
np.matmul(np.transpose(X), X)

array([[ 3.38000000e+02,  1.29893385e+00,  3.35035389e-01,
        -5.63367364e+00,  8.74994402e+00,  6.21964959e+00,
         8.24405374e+00,  5.87710487e+00,  1.47782711e+00],
       [ 1.29893385e+00,  7.74373631e+00, -2.50831412e-02,
         7.78906204e+00,  1.12178281e+00, -1.15089265e+00,
        -2.18436618e+00, -5.27985803e-01,  2.60682359e-01],
       [ 3.35035389e-01, -2.50831412e-02,  1.94945669e+00,
        -4.39571764e-01,  4.35128977e-01, -1.23382400e-01,
         9.48683121e-01,  2.80582310e-01, -3.47383812e-02],
       [-5.63367364e+00,  7.78906204e+00, -4.39571764e-01,
         6.85204836e+02,  1.12930630e+01,  3.56632915e+00,
         3.57415582e+00,  1.06198201e+01,  9.98439025e+00],
       [ 8.74994402e+00,  1.12178281e+00,  4.35128977e-01,
         1.12930630e+01,  4.82522748e+01, -1.57760501e+00,
        -2.58946057e-01, -7.94157899e-02, -1.03396081e+00],
       [ 6.21964959e+00, -1.15089265e+00, -1.23382400e-01,
         3.56632915e+00, -1.57760501e+00,  3.788640

In [74]:
np.linalg.inv(np.matmul(np.transpose(X), X))

array([[ 3.00635553e-03, -7.14078847e-04, -1.22832290e-04,
         5.68280831e-05, -5.65163621e-04, -4.98965467e-04,
        -4.94713928e-04, -4.08986404e-04, -2.22406219e-04],
       [-7.14078847e-04,  1.33707815e-01, -1.12038579e-03,
        -1.53003229e-03, -2.48629245e-03,  3.73471709e-03,
         6.68054314e-03,  9.01455561e-04, -1.63057461e-03],
       [-1.22832290e-04, -1.12038579e-03,  5.20077456e-01,
         4.77372445e-04, -4.71694164e-03,  2.28097219e-03,
        -1.14212128e-02, -1.55011561e-03,  1.19840860e-03],
       [ 5.68280831e-05, -1.53003229e-03,  4.77372445e-04,
         1.50304939e-03, -3.54150628e-04, -1.86512951e-04,
        -9.44861639e-05, -4.72129886e-04, -7.44457215e-04],
       [-5.65163621e-04, -2.48629245e-03, -4.71694164e-03,
        -3.54150628e-04,  2.10682175e-02,  8.85165976e-04,
         1.02834674e-04,  2.02718425e-04,  1.31929400e-03],
       [-4.98965467e-04,  3.73471709e-03,  2.28097219e-03,
        -1.86512951e-04,  8.85165976e-04,  2.676229

In [75]:
np.matmul(np.transpose(X), Y)

array([230.0, 2.6266859755600276, 0.6630343206510837, 123.40981606104879,
       16.131739967017236, 12.611916423348063, 13.942741609089923,
       11.32099836685913, 5.710250489890591], dtype=object)

In [76]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.6683100298919029, 0.09843964124158328, 0.15527361278369245,
       0.17588034169488304, 0.17894141702770094, 0.20347019592740487,
       0.14162264396230867, 0.12132201976156684, 0.14234816831377328],
      dtype=object)

In [77]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x+ B[6]*x+ B[7]*x+ B[8]*x
    return y


eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.668+ (0.098)x_1+ (0.155)x_2+ (0.176)x_3+ (0.179)x_4+ (0.203)x_5+ (0.142)x_6+ (0.121)x_7+ (0.142)x_8


In [78]:
dt_test = pd.read_pickle('Data_test.pickle')
dt_test = dt_test.drop('Gender', axis = 'columns')
dt_test = dt_test.drop('Married', axis = 'columns')
dt_test = dt_test.drop('Dependents', axis = 'columns')

attr = pd.read_pickle('Loan_attr.pickle')


In [79]:
# y_predict
dt_test['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['Self_Employed_WOE'] \
        + B[3]*dt_train['Credit_History_WOE'] + B[4]*dt_train['Property_Area_WOE'] + B[5]*dt_train['APPLICANTINCOME_BIN_WOE'] \
        + B[6]*dt_train['COAPPLICANTINCOME_BIN_WOE'] + B[7]*dt_train['LOANAMOUNT_BIN_WOE'] + B[8]*dt_train['LOAN_AMOUNT_TERM_BIN_WOE']

dt_test['P'] = 1/(1+np.exp(-dt_test['y_predict']))

# s_predict
dt_test['predict'] = dt_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
dt_test

,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,y_predict,P,predict
5,1,0,5417,4196.0,267.0,360.0,1.0,1,1,0.569754,0.638706,1
9,1,1,12841,10968.0,349.0,360.0,1.0,3,0,0.646188,0.656151,1
13,1,1,1853,2840.0,114.0,360.0,1.0,2,0,0.686266,0.665136,1
15,1,1,4950,0.0,125.0,360.0,1.0,1,1,0.728883,0.674560,1
16,0,1,3596,0.0,100.0,240.0,1.0,1,1,0.551611,0.634509,1
...,...,...,...,...,...,...,...,...,...,...,...,...
477,1,1,2873,1872.0,132.0,360.0,0.0,3,0,NaN,NaN,0
480,0,1,4350,0.0,154.0,360.0,1.0,2,1,0.724224,0.673536,1
482,1,1,2083,3150.0,128.0,360.0,1.0,3,1,NaN,NaN,0
488,1,0,4583,2083.0,160.0,360.0,1.0,3,1,0.843560,0.699214,1


In [80]:
n_correct = (dt_test['Loan_Status'] == dt_test['predict']).sum()
print(n_correct)

accuracy = n_correct/144
print('accuracy is')
print(accuracy)

91
accuracy is
0.6319444444444444
